# MLFlow Tracking Server Pruning Service

Before performing an installation you should familiarize yourself with below topics:

* [Pruning Service README](https://github.com/shapeandshare/anaconda.mlflow.tracking.prune/blob/main/README.md)
* [Install Guide](https://github.com/shapeandshare/anaconda.mlflow.tracking.prune/blob/main/docs/source/install_guide.md)


### 1. Create an instance of an AE Client

In [ ]:
from anaconda.enterprise.server.sdk.client import AEClient
from utils import get_ae_client

client: AEClient = get_ae_client()

### 2. Populate our service account secrets used for configuration (Part 1)

In [ ]:
from anaconda.enterprise.server.common.sdk import demand_env_var

# Define our secrets
secret_names: list[str] = ["MLFLOW_TRACKING_ENTITY_TTL"]

# If NOT deployed into the same account as the server then the following secrets MUST instead of the above be defined:
# secret_names: list[str] = ["MLFLOW_TRACKING_URI", "MLFLOW_REGISTRY_URI", "MLFLOW_TRACKING_TOKEN", "MLFLOW_TRACKING_ENTITY_TTL"]

# Remove possibly pre-existing configurations
for name in secret_names:
    try:
        client.secret_delete(key=name)
    except Exception as error:
        # This is broad to allow for catching and reporting everything.
        print(error)

    # [re]create the secrets using the latest values.
    value: str = demand_env_var(name=name)
    print(f"{name}:{value}")
    client.secret_put(key=name, value=value)

### 3. Upload Project

In [ ]:
from pathlib import Path
from anaconda.enterprise.server.contracts import ProjectUploadResponse

# Define the server template to upload
SERVER_TEMPLATE_PATH: str = Path("assets/anaconda.mlflow.tracking.prune-0.7.1.tar.gz").resolve().as_posix()

# Define the project name
SERVER_PROJECT_NAME: str = "test.mlflow.tracking.prune"

upload_response: ProjectUploadResponse = client.project_upload(
    project_archive_path=SERVER_TEMPLATE_PATH,
    name=SERVER_PROJECT_NAME,
)
upload_response

In [ ]:
project_revisions = client.project_revisions_get(project_id=upload_response.id)
project_revisions

In [ ]:
project_revisions

### 4. Create the Pruning Schedule

It is recommended to create a pruning schedule.  To do so use the ae5 tools.  The below will generate an ae5 command suitable for creating a basic schedule:

In [ ]:
job_creation_command: str = f"ae5 job create --command 'Prune' --schedule '*/10 * * * *' --name 'Scheduled {SERVER_PROJECT_NAME} Pruning' '{SERVER_PROJECT_NAME}'"
job_creation_command